In [75]:
class Experimentation(object):
    def __init__(self, variables, features_conf, regression_alg):
        
        self.variables = variables
        self.features_conf = features_conf
        self.regression_alg = regression_alg
        
    def fit(self, configuration__, mode):
        self.configuration__ = configuration__
        # declaring Bee
        self.geography = sb.Geography(configuration__, mode)
        self.sensors = sb.Sensors(configuration__, mode, path=configuration__['DATA_FOLDER'], delimit_geography=self.geography)
        self.features = sb.Features(configuration__, mode, Sensors=self.sensors, Geography=self.geography)
        
    def initiate(self, iterations, path_to_results):
        self.scores = []
        for var in self.variables:
            for fc in self.features_conf:
                for ra in self.regression_alg:
                    features = ['dow','day','week']
                    f = self.features.get_train_features(var)
                    if 'p' in fc:
                        features += [var]
                    if 'u' in fc:
                        features += self.configuration__['osm_line_objs']+self.configuration__['osm_point_objs']
                    if 's' in fc:
                        features += [x for x in self.configuration__['Sensors__variables'] if x!= var]
                    if 'd' in fc:
                        features += ['Index of Multiple Deprivation (IMD) Score', 'Income Score (rate)',
                                       'Employment Score (rate)', 'Education, Skills and Training Score',
                                       'Crime Score', 'Barriers to Housing and Services Score',
                                       'Living Environment Score',
                                       'Total population: mid 2012 (excluding prisoners)',
                                       'Population aged 16-59: mid 2012 (excluding prisoners)']

                    if ra == 'rf':
                        regressor = RandomForestRegressor(n_estimators=200, max_depth=5, max_features=1)
                    elif ra == 'gb':
                        regressor = GradientBoostingRegressor(n_estimators=200, max_depth=5, max_features=1)
                    
                    r2,mse = [],[]
                    for i in range(1,iterations+1):
                        m = sb.Model(regressor).fit(f['X'][features], f['y'])
                        r2.append(m.r2)
                        mse.append(m.mse)
                    self.scores.append([var,fc,ra,np.mean(r2),np.std(r2),np.mean(mse),np.std(mse)])
                    # print('[{} - {} - {}] r2: {} ~ std: {} --- features: {}'.format(var, fc, ra, np.mean(r2), np.std(r2), f['X'][features].columns))
        self.scores = pd.DataFrame(self.scores, columns=['variable','feature_configuration','regressor','r2','r2_std','mse','mse_std'])
        self.scores.to_csv(path_to_results)
        return self
                    

In [11]:
import sys
sys.path.append('/home/adelsondias/Repos/sensingbee/sensingbee')
import source as sb

configuration__ = {
    'DATA_FOLDER':'/home/adelsondias/Repos/newcastle/air-quality/data_1week1/',
    'SHAPE_PATH':'/home/adelsondias/Repos/newcastle/air-quality/shape/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales.shp',
    'Sensors__frequency':'D',
    'Sensors__variables': ['NO2','Temperature','PM2.5'],
    'Sensors__threshold_callibration': {'Temperature':25, 'NO2':80, 'PM2.5':15},
    'Geography__filter_column':'msoa11nm',
    'Geography__filter_label':'Newcastle upon Tyne',
    'Geography__meshgrid':{'dimensions':[50,50], 'longitude_range':[-1.8, -1.51], 'latitude_range':[54.96, 55.05]},
    'osm_bbox': '(54.96,-1.8,55.05,-1.51)',
    'osm_line_objs': ['primary','trunk','motorway','residential'],
    'osm_point_objs': ['traffic_signals','crossing']
}

In [70]:
e = Experimentation(variables=['NO2','PM2.5'], features_conf=['pusd','pus','p'], regression_alg=['gb'])

In [71]:
e.fit(configuration__, mode='load')
e.initiate(3,path_to_results='teste')

[NO2 - pusd - gb] r2: -2.9414761083263463 ~ std: 2.8608068695537963 --- features: Index(['dow', 'day', 'week', 'NO2', 'NO2', 'NO2', 'NO2', 'NO2', 'primary',
       'trunk', 'motorway', 'residential', 'traffic_signals', 'crossing',
       'Temperature', 'Temperature', 'Temperature', 'Temperature',
       'Temperature', 'PM2.5', 'PM2.5', 'PM2.5', 'PM2.5', 'PM2.5',
       'Index of Multiple Deprivation (IMD) Score', 'Income Score (rate)',
       'Employment Score (rate)', 'Education, Skills and Training Score',
       'Crime Score', 'Barriers to Housing and Services Score',
       'Living Environment Score',
       'Total population: mid 2012 (excluding prisoners)',
       'Population aged 16-59: mid 2012 (excluding prisoners)'],
      dtype='object')
[NO2 - pus - gb] r2: -0.7669343018640103 ~ std: 0.7366303368770928 --- features: Index(['dow', 'day', 'week', 'NO2', 'NO2', 'NO2', 'NO2', 'NO2', 'primary',
       'trunk', 'motorway', 'residential', 'traffic_signals', 'crossing',
       'Tem

In [72]:
e.scores

,variable,feature_configuration,regressor,r2,r2_std,mse,mse_std
0,NO2,pusd,gb,-2.941476,2.860807,4292.721466,249.938895
1,NO2,pus,gb,-0.766934,0.736630,3880.657987,325.901407
2,NO2,p,gb,-12.376472,4.526505,9055.962288,272.130082
3,PM2.5,pusd,gb,-1.062925,0.776470,20.167066,1.635735
4,PM2.5,pus,gb,-1.906519,0.676079,27.267305,1.120943
5,PM2.5,p,gb,-11.009121,1.127436,90.867878,8.744609
